# XARELLO Training - Colab v2

**IMPORTANT**: Before running:
1. Runtime → Change runtime type → **GPU** (T4)
2. This ensures you get ~12GB RAM + GPU acceleration

## Step 1: Check resources

In [ ]:
import torch
import psutil
import sys

print(f"Python: {sys.version}")
print(f"RAM: {psutil.virtual_memory().total / 1e9:.1f} GB")
print(f"GPU: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

if psutil.virtual_memory().total / 1e9 < 10:
    print("\n⚠️  WARNING: Less than 10GB RAM. Enable GPU runtime for more memory!")
    print("Go to: Runtime → Change runtime type → GPU")

## Step 2: Install all dependencies first

In [ ]:
# Install specific compatible versions
!pip uninstall -y transformers huggingface-hub peft -q
!pip install transformers==4.38.1 huggingface-hub==0.21.0 peft==0.9.0 -q
!pip install gymnasium OpenAttack datasets -q

# Verify
import transformers, peft, huggingface_hub
print(f"transformers: {transformers.__version__}")
print(f"peft: {peft.__version__}")
print(f"huggingface_hub: {huggingface_hub.__version__}")

## Step 3: Clone repositories

In [ ]:
!rm -rf BODEGA xarello  # Clean any previous clones

# Clone your forks with the fixes
!git clone -b playground https://github.com/marti-farre/BODEGA.git
!git clone -b testing-stuff https://github.com/marti-farre/xarello.git

print("\nCloned successfully.")

## Step 4: Upload data files

Upload `dev.tsv` and `BiLSTM-512.pth`

In [ ]:
import os
from google.colab import files

# Create directories
os.makedirs('/root/data/BODEGA/RD', exist_ok=True)
os.makedirs('/root/data/xarello/models/wide/RD-BiLSTM', exist_ok=True)
os.environ['HOME'] = '/root'

print("Upload dev.tsv and BiLSTM-512.pth:")
uploaded = files.upload()

for filename in uploaded.keys():
    if 'dev' in filename.lower():
        os.rename(filename, '/root/data/BODEGA/RD/dev.tsv')
        print(f"✓ {filename} → /root/data/BODEGA/RD/dev.tsv")
    elif filename.endswith('.pth'):
        os.rename(filename, '/root/data/BODEGA/RD/BiLSTM-512.pth')
        print(f"✓ {filename} → /root/data/BODEGA/RD/BiLSTM-512.pth")

print("\nFiles ready:")
!ls -la /root/data/BODEGA/RD/

import sys
import subprocess

if sys.version_info >= (3, 12):
    print("Patching OpenAttack for Python 3.12...")
    
    # Find OpenAttack path WITHOUT importing it
    result = subprocess.run([sys.executable, '-c', 
        'import importlib.util; spec = importlib.util.find_spec("OpenAttack"); print(spec.submodule_search_locations[0])'],
        capture_output=True, text=True)
    oa_path = result.stdout.strip()
    
    if not oa_path:
        # Fallback: find in site-packages
        import site
        for sp in site.getsitepackages():
            candidate = f"{sp}/OpenAttack"
            import os
            if os.path.exists(candidate):
                oa_path = candidate
                break
    
    print(f"OpenAttack path: {oa_path}")
    data_init = f"{oa_path}/data/__init__.py"
    
    with open(data_init, 'r') as f:
        content = f.read()
    
    # Replace the problematic line
    old_code = "data = data.module_finder.find_loader(data.name)[0].load_module()"
    new_code = """loader = importlib.util.find_spec(data.name, data.module_finder.path).loader
            data = loader.load_module(data.name)"""
    
    if old_code in content:
        # Also need to add import at top
        if "import importlib.util" not in content:
            content = "import importlib.util\n" + content
        content = content.replace(old_code, new_code)
        with open(data_init, 'w') as f:
            f.write(content)
        print("✓ OpenAttack patched successfully")
    else:
        print("OpenAttack already patched or different version")
else:
    print(f"Python {sys.version_info.major}.{sys.version_info.minor} - no patch needed")

In [ ]:
import sys
if sys.version_info >= (3, 12):
    print("Patching OpenAttack for Python 3.12...")
    
    # Find OpenAttack data/__init__.py
    import OpenAttack
    oa_path = OpenAttack.__path__[0]
    data_init = f"{oa_path}/data/__init__.py"
    
    with open(data_init, 'r') as f:
        content = f.read()
    
    # Replace the problematic code
    old_code = "data = data.module_finder.find_loader(data.name)[0].load_module()"
    new_code = """import importlib.util
            spec = importlib.util.find_spec(data.name)
            data = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(data)"""
    
    if old_code in content:
        content = content.replace(old_code, new_code)
        with open(data_init, 'w') as f:
            f.write(content)
        print("✓ OpenAttack patched successfully")
    else:
        print("OpenAttack already patched or different version")
else:
    print(f"Python {sys.version_info.major}.{sys.version_info.minor} - no patch needed")

## Step 6: Test imports

In [ ]:
import sys
sys.path.insert(0, '/content/BODEGA')

# Test critical imports
try:
    from victims.bilstm import VictimBiLSTM
    print("✓ VictimBiLSTM imported")
except Exception as e:
    print(f"✗ VictimBiLSTM failed: {e}")

try:
    from victims.transformer import VictimTransformer
    print("✓ VictimTransformer imported")
except Exception as e:
    print(f"✗ VictimTransformer failed: {e}")

try:
    import OpenAttack
    print("✓ OpenAttack imported")
except Exception as e:
    print(f"✗ OpenAttack failed: {e}")

print("\nAll imports OK!")

## Step 7: Train XARELLO

Full training with GPU - should take 1-2 hours

In [ ]:
import os
os.chdir('/content/xarello')
os.environ['HOME'] = '/root'
os.environ['MPLBACKEND'] = 'agg'  # Fix matplotlib backend

!PYTHONPATH=/content/BODEGA python main-train-eval.py RD BiLSTM /root/data/xarello/models/wide/RD-BiLSTM

## Step 8: Download model

In [ ]:
from google.colab import files
import os

model_dir = '/root/data/xarello/models/wide/RD-BiLSTM'
model_path = f'{model_dir}/xarello-qmodel.pth'

print("Model directory contents:")
!ls -la {model_dir}

if os.path.exists(model_path):
    print(f"\nDownloading model ({os.path.getsize(model_path)/1e6:.1f} MB)...")
    files.download(model_path)
else:
    print("\n⚠️  Model not found. Check training output above.")

## Step 9: Download plots (optional)

In [ ]:
from google.colab import files
import glob

model_dir = '/root/data/xarello/models/wide/RD-BiLSTM'
for pdf in glob.glob(f'{model_dir}/*.pdf'):
    print(f"Downloading {os.path.basename(pdf)}...")
    files.download(pdf)

---
## Done!

Place `xarello-qmodel.pth` at `~/data/xarello/models/wide/RD-BiLSTM/` locally and run evaluation.